![car](car.jpg)

Insurance companies invest a lot of [time and money](https://www.accenture.com/_acnmedia/pdf-84/accenture-machine-leaning-insurance.pdf) into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries insurance it is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

Knowing all of this, On the Road car insurance have requested your services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked you to identify the single feature that results in the best performing model, as measured by accuracy, so they can start with a simple model in production.

They have supplied you with their customer data as a csv file called `car_insurance.csv`, along with a table detailing the column names and descriptions below.



## The dataset

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-25</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code | 
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client | 
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

In [115]:
# Import required modules
import pandas as pd
import numpy as np
from statsmodels.formula.api import logit

In [116]:
# Read data
data = pd.read_csv('car_insurance.csv')
print('Data preview:')
print(data.head())
print("-------------------------------------------------------------")
print('Data types:')
print(data.info())
print("-------------------------------------------------------------")
print('Data shape:')
print(data.shape)

Data preview:
       id  age  gender  ... duis past_accidents outcome
0  569520    3       0  ...    0              0     0.0
1  750365    0       1  ...    0              0     1.0
2  199901    0       0  ...    0              0     0.0
3  478866    0       1  ...    0              0     0.0
4  731664    1       1  ...    0              1     1.0

[5 rows x 18 columns]
-------------------------------------------------------------
Data types:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10000 non-null  int64  
 1   age                  10000 non-null  int64  
 2   gender               10000 non-null  int64  
 3   driving_experience   10000 non-null  object 
 4   education            10000 non-null  object 
 5   income               10000 non-null  object 
 6   credit_score         9018 non-null   flo

In [117]:
# Check missing values
print(data.isna().sum())
missing_value_cols = list(data.columns[data.isna().any()])
print('\nColumn {} has missing values.'.format(tuple(missing_value_cols)))

# Fill missing values with their column's mean
filled_data = data.fillna({missing_value_cols[0]:data[missing_value_cols[0]].mean(),
                           missing_value_cols[1]:data[missing_value_cols[1]].mean()})

# Check y data if is binary
print("-------------------------------------------------------------\nY data:")
print(data['outcome'].value_counts())

id                       0
age                      0
gender                   0
driving_experience       0
education                0
income                   0
credit_score           982
vehicle_ownership        0
vehicle_year             0
married                  0
children                 0
postal_code              0
annual_mileage         957
vehicle_type             0
speeding_violations      0
duis                     0
past_accidents           0
outcome                  0
dtype: int64

Column ('credit_score', 'annual_mileage') has missing values.
-------------------------------------------------------------
Y data:
0.0    6867
1.0    3133
Name: outcome, dtype: int64


In [118]:
# Modeling
models = []
features = [col for col in data.columns if col != 'id' and col != 'outcome'] # excluding the 'id' and 'outcome' column
for i in features:
    mdl = logit('outcome ~ {}'.format(i), data = filled_data).fit()
    models.append(mdl)

Optimization terminated successfully.
         Current function value: 0.511794
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.615951
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.467092
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.603742
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.531499
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.572557
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.552412
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.572668
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.586659
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.595431
  

In [119]:
# Accessing
def accuracy(TN, FP, FN, TP):
    acc = (TN + TP) / (TN + FP + FN + TP)
    return acc
accuracies = [] # init
TN, FP, FN,TP = 0, 0, 0, 0 # init
for model in models:
    cf_mtx = model.pred_table()
    TN = cf_mtx[0,0]
    FP = cf_mtx[0,1]
    FN = cf_mtx[1,0]
    TP = cf_mtx[1,1]
    accuracies.append(accuracy(TN = TN, FP = FP, FN = FN, TP = TP))
fea_acc = pd.DataFrame({'best_feature':features, 'best_accuracy': accuracies})
print(fea_acc)
idx = accuracies.index(max(accuracies))
best_feature_df = pd.DataFrame([fea_acc.iloc[idx,]])
print("-------------------------------------------------------------")
print(best_feature_df)

           best_feature  best_accuracy
0                   age         0.7747
1                gender         0.6867
2    driving_experience         0.7771
3             education         0.6867
4                income         0.7425
5          credit_score         0.7054
6     vehicle_ownership         0.7351
7          vehicle_year         0.6867
8               married         0.6867
9              children         0.6867
10          postal_code         0.6867
11       annual_mileage         0.6904
12         vehicle_type         0.6867
13  speeding_violations         0.6867
14                 duis         0.6867
15       past_accidents         0.6867
-------------------------------------------------------------
         best_feature  best_accuracy
2  driving_experience         0.7771
